In [1]:
import numpy as np
import pandas as pd
import itertools
from Bio import SeqIO
from tqdm.auto import tqdm

import joblib

## Wczytanie i przygotowanie danych

### Pliki `.bed`

In [2]:
negative_set = pd.read_csv('negative_set.bed', delimiter='\t', header=None, names=["Chromosome", "StartPos", "EndPos"])
positive_set = pd.read_csv('GM12878.bed', delimiter='\t', header=None, names=["Chromosome", "StartPos", "EndPos", "Score"])

In [3]:
negative_set.head(5)

,Chromosome,StartPos,EndPos
0,chr10,32251,36771
1,chr10,39431,39891
2,chr10,72312,74222
3,chr10,84717,85177
4,chr10,90499,91949


In [4]:
positive_set.head(5)

,Chromosome,StartPos,EndPos,Score
0,chr1,773300,774100,7.866088
1,chr1,778980,779450,6.472419
2,chr1,800100,802000,11.010675
3,chr1,825670,826410,6.114487
4,chr1,839470,842590,8.848865


In [5]:
# dodanie etykiety do zbiorów
negative_set["IsEnhancer"] = 0
positive_set["IsEnhancer"] = 1

positive_set = positive_set.drop(columns="Score")

In [6]:
# połączenie zbiorów danych
total_set = pd.concat([negative_set, positive_set])

# usunięcie nieprawidłowych chromosomów
total_set = total_set.loc[total_set["Chromosome"].str.match(r"chr\d+")]

total_set = total_set.reset_index(drop=True)

In [7]:
total_set

,Chromosome,StartPos,EndPos,IsEnhancer
0,chr10,32251,36771,0
1,chr10,39431,39891,0
2,chr10,72312,74222,0
3,chr10,84717,85177,0
4,chr10,90499,91949,0
...,...,...,...,...
127034,chr9,140651150,140652330,1
127035,chr9,140702130,140703100,1
127036,chr9,140703310,140704120,1
127037,chr9,140710520,140711890,1


#### Średnia długość enhancera!!!

In [8]:
total_set["Length"] = total_set["EndPos"] - total_set["StartPos"]
total_set.loc[total_set["IsEnhancer"] == 1, "Length"].mean()

# średnia długość to 1700!!

1744.0801070840198

### Plik `.fasta`

In [9]:
# odczyt pliku za pomocą biblioteki biopython
plik = "GRCh37.primary_assembly.genome.fa"

fasta_sequences = {}

with open(plik, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        fasta_sequences[record.id] = record.seq

In [10]:
WINDOW = 1700

records = []

for chromosome, sequence in fasta_sequences.items():

    # pomijamy dziwne chromosomy
    if chromosome not in total_set["Chromosome"].unique():
        continue

    for idx, pos in enumerate(range(0, len(sequence), WINDOW)):
        records.append({
            "Chromosome": chromosome,
            "idx": idx+1,
            "startPos": pos,
            "endPod": pos + WINDOW,
            "DNA sequence": str(sequence[pos:pos+WINDOW])
        })

data = pd.DataFrame(records)

# flaga pokazująca czy sekwencja DNA nie zawiera za dużo 'N'
data["IsValid"] = (data["DNA sequence"].str.count("N") / data["DNA sequence"].str.len()).le(0.05).astype(int)

# usuwamy 'N' z sekwencji, kiedy 'N' jest co najwyżej 5%
data.loc[data["IsValid"] == 1, "DNA sequence"] = data.loc[data["IsValid"] == 1, "DNA sequence"].str.replace("N", "")
valid_set = data.loc[data["IsValid"] == 1, :]

### k-mery

In [11]:
# funkcja do zliczenia k-merów w sekwencji DNA
# dla całej ramki sett
# i do zapisu zliczonych częstości do pliku
def count_and_save_kmers(k, sett):

    # lista wszystkich k-merów
    kmers = [''.join(kmer_tuple) for kmer_tuple in itertools.product('ACTG', repeat=k)]

    # funkcja do odwracania k-merów
    def reverse_kmer(seq: str):
        mapping = {
            "A": "T",
            "T": "A",
            "C": "G",
            "G": "C",
        }
        return "".join([mapping[s] for s in seq[::-1]])

    # lista unikalnych k-merów, które (i których odwrócenia) będziemy zliczać
    # dla k=4 liczy 136 elementów
    unique_kmers = []
    for kmer in kmers:
        reversed_kmer = reverse_kmer(kmer)
        if kmer not in unique_kmers and reversed_kmer not in unique_kmers:
            unique_kmers.append(kmer)

    # słownik do zliczania k-merów
    kmers_counts = {kmer: pd.Series(index=sett.index, data=0) for kmer in unique_kmers}

    # zliczamy k-mery
    for kmer in tqdm(kmers):
        if kmer in unique_kmers:  
            kmers_counts[kmer] += sett["DNA sequence"].str.count(kmer)
        else:
            reversed_kmer = reverse_kmer(kmer)
            kmers_counts[reversed_kmer] = sett["DNA sequence"].str.count(kmer)

    # łączymy częstości w ramkę danych
    kmers_counts_df = pd.concat(kmers_counts, axis=1).round(6)

    # dzielimy przez długość sekwencji
    kmers_counts_df = kmers_counts_df.div(sett["DNA sequence"].str.len(), axis=0)

    # łączymy z pierwotną ramką sett i zapisujemy do pliku 
    sett_with_kmers = pd.concat([sett, kmers_counts_df], axis=1)
    (
        sett_with_kmers
            .drop(columns=["DNA sequence"])
            .to_parquet(f"data_{k}_mer.gzip", compression="gzip", index=False)
    )

In [12]:
# zliczamy częstości k-merów dla kilku wybranych k

# count_and_save_kmers(4, valid_set)

## Predykcja

### Wczytanie modelu

In [13]:
model = joblib.load("model.pkl")
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

### Predykcja

In [14]:
data_4_mer = pd.read_parquet(f"data_4_mer.gzip")
X = data_4_mer.drop(columns=["Chromosome", "startPos", "endPod", "IsValid"]) # + "idx"
proba_predicted =  model.predict_proba(X)

In [15]:
# 1 == Enhancer
model.classes_

array([0, 1], dtype=int64)

In [16]:
# druga kolumna to zatem prawdopodobieństwo bycia enhancerem
# przypisujemy do ramki z oknami DNA

data.loc[data["IsValid"] == 1, "EnhancerProbability"] = proba_predicted[:, 1]

In [17]:
# średnia predykcja dla każdego enhancera
mean_prediction = data.loc[data["IsValid"] == 1, :].groupby("Chromosome", as_index=False).agg(mean_prediction=("EnhancerProbability", "mean"))

In [18]:
# dodanie średniej predykcji dla chromosomu do ramki
data = pd.merge(data, mean_prediction, on="Chromosome", how="inner")

# tam gdzie nie było predykcji z modelu (dużo N) dajemy średnią predykcję
data.loc[data["IsValid"] == 0, "EnhancerProbability"] = data.loc[data["IsValid"] == 0, "mean_prediction"]

In [20]:
# zapisanie finalnych danych, które zostaną wstawione do bazy
data["EnhancerProbability"] = data["EnhancerProbability"].round(3)
data = data.reset_index()
data["index"] += 1

data.loc[:, [
    "index",
    "Chromosome",
    "idx",
    "startPos",
    "endPod",
    "EnhancerProbability"
]].rename(columns={
    "index": "id",
    "Chromosome": "chromosome",
    "idx": "window_index",
    "startPos": "start_position",
    "endPod": "end_position",
    "EnhancerProbability": "enhancer_probability"
}).to_parquet("final_data_to_database.gzip", index=False, compression="gzip")

## Wstawienie danych do bazy

In [22]:
final_data_to_database = pd.read_parquet("final_data_to_database.gzip")

In [25]:
import sqlite3
conn = sqlite3.connect('C://Users/User/Documents/IiAD/Semestr VI/Warsztaty badawcze 2/wb-django-service/db.sqlite3')
final_data_to_database.to_sql("service_dnawindow", conn, if_exists="replace", index=False)
conn.close()